In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)


train.shape, test.shape, val.shape

((47520, 41), (14358, 40), (11880, 41))

In [4]:
import numpy as np



def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    # Also create a "missing indicator" column, because the fact that
    # values are missing may be a predictive signal.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
            
    # Drop duplicate columns
    duplicates = ['quantity_group', 'payment_type']
    X = X.drop(columns=duplicates)
    
    # Drop recorded_by (never varies) and id (always varies, random)
    unusable_variance = ['recorded_by', 'id']
    X = X.drop(columns=unusable_variance)
    
    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    # Engineer feature: how many years from construction_year to date_recorded
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()
    
    # return the wrangled dataframe
    return X

In [5]:
train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [6]:
target = 'status_group'

train_features = train.drop(columns=[target])

numeric_features = train_features.select_dtypes(include='number').columns.tolist()

cardinality = train_features.select_dtypes(exclude='number').nunique()

categorical_features = cardinality[cardinality <= 50].index.tolist()

features = numeric_features + categorical_features

In [15]:
X_train

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,year_recorded,...,source_type,source_class,waterpoint_type,waterpoint_type_group,longitude_MISSING,latitude_MISSING,construction_year_MISSING,gps_height_MISSING,population_MISSING,years_MISSING
43360,0.0,NaN,33.542898,-9.174777,0,12,4,NaN,NaN,2011,...,spring,groundwater,communal standpipe,communal standpipe,False,False,True,True,True,True
7263,500.0,2049.0,34.665760,-9.308548,0,11,4,175.0,2008.0,2011,...,spring,groundwater,communal standpipe,communal standpipe,False,False,False,False,False,False
2486,25.0,290.0,38.238568,-6.179919,0,6,1,2300.0,2010.0,2011,...,shallow well,groundwater,hand pump,hand pump,False,False,False,False,False,False
313,0.0,NaN,30.716727,-1.289055,0,18,1,NaN,NaN,2011,...,shallow well,groundwater,other,other,False,False,True,True,True,True
52726,0.0,NaN,35.389331,-6.399942,0,1,6,NaN,NaN,2011,...,borehole,groundwater,communal standpipe,communal standpipe,False,False,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,50.0,489.0,38.268574,-5.450254,4,4,6,1300.0,2009.0,2011,...,river/lake,surface,communal standpipe,communal standpipe,False,False,False,False,False,False
58170,0.0,NaN,33.926294,-9.641293,0,12,3,NaN,NaN,2011,...,spring,groundwater,communal standpipe,communal standpipe,False,False,True,True,True,True
17191,0.0,599.0,39.262924,-10.768079,0,90,33,1.0,2012.0,2013,...,borehole,groundwater,communal standpipe multiple,communal standpipe,False,False,False,False,False,False
8192,30.0,426.0,39.348550,-10.642069,0,90,33,320.0,1988.0,2013,...,borehole,groundwater,communal standpipe multiple,communal standpipe,False,False,False,False,False,False
